In [1]:
import pandas as pd
import numpy as np
import cv2

# testデータ読み込み

In [2]:
voice = pd.read_pickle("../../voice/train/VCC2SF1/10002.pkl")
image = cv2.imread("../../image/000001.jpg")

# ノーマライズ

## 音声

min = 0, max = 1 に変換

In [3]:
def norm_voice(array):
    min = array.min()
    array -= min 
    
    max = array.max()
    array /= max
    
    return array

In [4]:
# test
voice = norm_voice(voice)

入力値のuttr_lenが4の倍数じゃないとバグるので最終を切る

In [5]:
voice = voice[:, :voice.shape[1]-voice.shape[1]%4]

## 画像

In [6]:
def norm_image(array):
    return array/255

In [7]:
image = norm_image(image)

# 学習テスト

In [8]:
import cmvc.models.model as cmvc
import torch
from torch import optim

In [9]:
device = "cpu"
net = cmvc.Net()

"""

入力

1: バッチサイズ

2: channel

3: mccs_size(画像における縦)

4: uttr_len(画像における横)

"""

In [10]:
x = torch.tensor([[voice], [voice]], dtype=torch.float32)

In [11]:
y = torch.tensor([image.T, image.T], dtype=torch.float32)

In [12]:
x.shape

torch.Size([2, 1, 36, 176])

In [13]:
y.shape

torch.Size([2, 3, 32, 32])

In [35]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.train()

x = x.to(device) 
y = y.to(device)
net.zero_grad()
#print(net.test(x, y))
loss = net.loss(x, y)
print(loss)
loss.backward()
optimizer.step()

tensor(5.7689, grad_fn=<SumBackward0>) tensor(699.7460, grad_fn=<SumBackward0>)
tensor(0.0118, grad_fn=<SumBackward0>) tensor(16.0012, grad_fn=<SumBackward0>)
tensor(1.3289, grad_fn=<MulBackward0>) tensor(1.3530e-05, grad_fn=<MulBackward0>) tensor(0.4314, grad_fn=<MseLossBackward>) tensor(0.1289, grad_fn=<MseLossBackward>)
tensor(0.1290, grad_fn=<AddBackward0>)
tensor(2.0182, grad_fn=<AddBackward0>)
